<a href="https://colab.research.google.com/github/mattyco/Closed-Domain-Question-Answering-System/blob/master/cdqa_proj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Initializations
Creating/Installing

In [0]:
!pip install cdqa #installs the cdqa library using pip3

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import os
import pandas as pd
from ast import literal_eval

from cdqa.utils.filters import filter_paragraphs
from cdqa.pipeline import QAPipeline

/usr/local/lib/python3.6/dist-packages/tqdm/autonotebook/__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [0]:
from cdqa.utils.download import download_model

download_model(model='bert-squad_1.1', dir='./models')      #downloads the pretrained model that was built on that news articles dataset


bert_qa.joblib already downloaded


In [0]:
from google.colab import files
uploaded = files.upload()         #calls ui to upload the dataset as csv

Saving dataset.csv to dataset.csv


In [0]:
df = pd.read_csv('./dataset.csv', converters={'paragraphs': literal_eval},encoding="Latin-1", names=['title','paragraphs'], header=None)
df = filter_paragraphs(df)
df.head()                           #dataset is read and converted into a pandas dataframe

,title,paragraphs
0,Chapter One ELECTRIC CHARGES AND FIELDS,[1.1 INTRODUCTION All of us have the experienc...
1,Chapter Two ELECTROSTATIC POTENTIAL AND CAPACI...,[2.1 INTRODUCTION In Chapters 6 and 8 (Class X...
2,Chapter Three CURRENT ELECTRICITY,"[3.1 INTRODUCTION In Chapter 1, all charges wh..."
3,Chapter Four MOVING CHARGES AND MAGNETISM,[4.1 INTRODUCTION Both Electricity and Magneti...
4,Chapter Five MAGNETISM AND MATTER,[5.1 INTRODUCTION Magnetic phenomena are unive...


#Retriever Training

In [0]:
cdqa_pipeline = QAPipeline(reader='./models/bert_qa.joblib')
cdqa_pipeline.fit_retriever(df=df)                                #calls the reader  and retriever part of the cdqa lib that parses the dataset

#retriever takes a pool of paragraphs as input and ranks and scores them base

100%|██████████| 231508/231508 [00:00<00:00, 5733978.94B/s]


QAPipeline(reader=BertQA(adam_epsilon=1e-08, bert_model='bert-base-uncased',
                         do_lower_case=True, fp16=False,
                         gradient_accumulation_steps=1, learning_rate=5e-05,
                         local_rank=-1, loss_scale=0, max_answer_length=30,
                         n_best_size=20, no_cuda=False,
                         null_score_diff_threshold=0.0, num_train_epochs=3.0,
                         output_dir=None, predict_batch_size=8, seed=42,
                         server_ip='', server_po...size=8,
                         verbose_logging=False, version_2_with_negative=False,
                         warmup_proportion=0.1, warmup_steps=0),
           retrieve_by_doc=False,
           retriever=BM25Retriever(b=0.75, floor=None, k1=2.0, lowercase=True,
                                   max_df=0.85, min_df=2, ngram_range=(1, 2),
                                   preprocessor=None, stop_words='english',
                                   t

#Reader Training

In [0]:
#training the reader

#uploaded = files.upload()
cdqa_pipeline.fit_reader('dataset.json')
cdqa_pipeline.dump_reader('./models/bert_qa.joblib')


Could not find answer: '' vs. 'Electric Charges and Fields neutralise or nullify each others effect'
Could not find answer: '' vs. 'the charge on glass rod or cat's fur'
Could not find answer: '' vs. 'the charge on plastic rod or silk'
Could not find answer: '' vs. 'Coulomb's law gives the force between two charges q1 and q2 in vacuum'
Could not find answer: '' vs. 'the principle of superposition says that in a system of charges q1, q2, ..., qn, the force on q1 due to q2 is the same as given by Coulomb's law, i.e., it is unaffected by the presence of the other charges q3, q4, ..., qn. The total force F1 on the charge q1, due to all other charges, is then given by the vector sum of the forces F12, F13, ..., F1n'
Could not find answer: '' vs. 'Coulomb's law is consistent with Newton's third law'
Could not find answer: '' vs. 'An electric dipole is a pair of equal and opposite point charges , separated by a distance 2a'
Could not find answer: '' vs. 'The electric field at any general poin

#Testing

In [0]:
query = "What is voltage?"
prediction = cdqa_pipeline.predict(query, n_predictions=5)

In [0]:

print('query: {}'.format(query))
for ans in prediction:
  print('answer: {}'.format(ans[0]))
  print('title: {}'.format(ans[1]))
  print('paragraph: {}'.format(ans[2]))
  print("\n")

query: What is voltage?
answer: The current through the capacitor is Ï/2 ahead of the applied voltage.
title: Chapter Seven ALTERNATING CURRENT 
paragraph: The current through the capacitor is Ï/2 ahead of the applied voltage. As in the case of inductor, the average power supplied to a capacitor over one complete cycle is zero. For a series RLC circuit driven by voltage v = vm sinÏt, the current is given by i = im sin (Ït + Ï ) where


answer: When an alternating voltage is applied to the primary, the resulting current produces an alternating magnetic flux which links the secondary and induces an emf in it
title: Chapter Seven ALTERNATING CURRENT 
paragraph: When an alternating voltage is applied to the primary, the resulting current produces an alternating magnetic flux which links the secondary and induces an emf in it. The value of this emf depends on the number of turns in the secondary. We consider an ideal transformer in which the primary has negligible resistance and all th

In [0]:
from cdqa.utils.evaluation import evaluate_reader

evaluate_reader(cdqa_pipeline, 'dataset.json')

Evaluation expects v-1.1, but got dataset with v-v1.1


KeyboardInterrupt: ignored

In [0]:
from cdqa.utils.evaluation import evaluate_pipeline

evaluate_pipeline(cdqa_pipeline, 'dataset.json')



Evaluation results: {'exact_match': 43.728813559322035, 'f1': 59.32708985762988}


{'exact_match': 43.728813559322035, 'f1': 59.32708985762988}

In [0]:
# from cdqa.utils.converters import df2squad
# # Converting dataframe to SQuAD format
# json_data = df2squad(df=df, squad_version='v1.1', output_dir='.', filename='dataset')
# files.download("dataset.json")